# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob
import sys
sys.path.append("../05_src")
#sys.path.append(os.getenv('SRC_DIR')) 

# Write your code below.
PRICE_DATA = os.getenv('PRICE_DATA')

parquet_files = glob(os.path.join("/Users/saranyamanoharan/DSI/production/05_src/data/prices", "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")


In [27]:
print(dd_shift)

Dask DataFrame Structure:
                          Date     Open     High      Low    Close Adj Close   Volume  source   Year Adj_close_lag_1
npartitions=90                                                                                                      
A               datetime64[ns]  float64  float64  float64  float64   float64  float64  object  int32         float64
ACB                        ...      ...      ...      ...      ...       ...      ...     ...    ...             ...
...                        ...      ...      ...      ...      ...       ...      ...     ...    ...             ...
ZEUS                       ...      ...      ...      ...      ...       ...      ...     ...    ...             ...
ZEUS                       ...      ...      ...      ...      ...       ...      ...     ...    ...             ...
Dask Name: lambda, 9 graph layers


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [49]:
# Write your code below.
dd_shift_1 = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1)))

/var/folders/24/g41x4rxn4gzdvbhcvk59qnqc0000gn/T/ipykernel_7755/897312482.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift_1 = dd_px.groupby('ticker', group_keys=False).apply(


In [50]:

dd_shift_2 = dd_shift_1.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Adj_close_lag_1 = x['Adj Close'].shift(1))
)

/var/folders/24/g41x4rxn4gzdvbhcvk59qnqc0000gn/T/ipykernel_7755/1708443291.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift_2 = dd_shift_1.groupby('ticker', group_keys=False).apply(


In [51]:
dd_returns = dd_shift_2.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

In [ ]:
dd_feat = dd_returns.assign(
    hi_lo_range = lambda x: x['High']-x['Low']
)

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_close_lag_1,Returns,hi_lo_range
ticker,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,-0.082386,2.280043
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,0.089783,2.816525
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,-0.090909,2.592991
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,0.026563,1.385908
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,2020-03-26,9.610000,9.940000,9.260000,9.590000,9.590000,60500.0,ZEUS.csv,2020,9.670000,9.670000,-0.008273,0.679999
ZEUS,2020-03-27,9.330000,9.330000,8.700000,8.700000,8.700000,52900.0,ZEUS.csv,2020,9.590000,9.590000,-0.092805,0.630000
ZEUS,2020-03-30,8.810000,9.760000,8.700000,9.680000,9.680000,73700.0,ZEUS.csv,2020,8.700000,8.700000,0.112644,1.060000


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [60]:
# Write your code below.
dd_feat_df = dd_feat.compute()
type(dd_feat_df)

pandas.core.frame.DataFrame

In [63]:

dd_feat_df["returns_10"] = dd_feat_df["Returns"].rolling(10).mean() 
dd_feat_df.head(20)

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_close_lag_1,Returns,hi_lo_range,returns_10
ticker,,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078,NaN
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,-0.082386,2.280043,NaN
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,0.089783,2.816525,NaN
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,-0.090909,2.592991,NaN
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,0.026563,1.385908,NaN
A,1999-11-26,29.238197,29.685265,29.148785,29.461731,25.338428,1729400.0,A.csv,1999,29.372318,25.261524,0.003044,0.536480,NaN
A,1999-11-29,29.327610,30.355865,29.014664,30.132332,25.915169,4074700.0,A.csv,1999,29.461731,25.338428,0.022762,1.341202,NaN
A,1999-11-30,30.042919,30.713520,29.282904,30.177038,25.953619,4310000.0,A.csv,1999,30.132332,25.915169,0.001484,1.430616,NaN
A,1999-12-01,30.177038,31.071173,29.953505,30.713520,26.415012,2957300.0,A.csv,1999,30.177038,25.953619,0.017778,1.117668,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it was not necessary to convert to pandas to calculate the moving average return. However, pandas is recommended since it's not a resource-intensive calculation.  

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.